In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

# Load your case data
df_cases = pd.read_csv('Harris_EV_Filings_2023_042025_CLEAN.csv', usecols=['Case Number', 'court_number'])
sampled_cases = df_cases.dropna(subset=['Case Number']).sample(n=10, random_state=42)

session = requests.Session()

def scrape_eviction_case(case_number, court_number):
    url = f'https://jpwebsite.harriscountytx.gov/CaseInfo/GetCaseInfo?case={case_number}'
    for attempt in range(3):
        try:
            response = session.get(url, timeout=10)
            response.raise_for_status()
            break
        except requests.RequestException as e:
            if attempt == 2:
                return {'Case Number': case_number, 'court_number': court_number, 'Error': str(e)}
            time.sleep(1)

    soup = BeautifulSoup(response.text, 'html.parser')
    result = {'Case Number': case_number, 'court_number': court_number}

    def extract_field(label):
        tag = soup.find('span', string=label)
        if tag and tag.find_next_sibling('span'):
            return tag.find_next_sibling('span').text.strip()
        return ''

    result['Filed Date'] = extract_field('Filed Date:')
    result['Case Status'] = extract_field('Case Status:')
    result['Nature of Claim'] = extract_field('Nature of Claim:')
    result['Disposition Desc'] = extract_field('Disposition:')
    result['Disposition Date'] = extract_field('Disposition Date:')
    result['Judgment Date'] = extract_field('Judgment Date:')
    result['Claim Amount'] = extract_field('Claim Amount:')

    # Parties
    party_blocks = soup.select('#partyInfo div.even, #partyInfo div.odd')
    for block in party_blocks:
        role = block.find('span', string='Party Type:')
        name = block.find('span', string='Party Name:')
        if role and name:
            role_value = role.find_next_sibling('span').text.strip()
            name_value = name.find_next_sibling('span').text.strip()
            if role_value == 'Plaintiff':
                result['Plaintiff'] = name_value
            elif role_value == 'Defendant':
                result['Defendant'] = name_value

    if not result.get('Filed Date') or not result.get('Plaintiff'):
        result['Error'] = 'Missing core data'
        return result

    # Hearings (up to 10)
    hearing_blocks = soup.select('#eventInfo div.even, #eventInfo div.odd')[:10]
    for i, block in enumerate(hearing_blocks):
        desc = block.find('span', string='Hearing Description:')
        date = block.find('span', string='Hearing Date/Time:')
        result_text = block.find_all('span')[-3].text.strip() if len(block.find_all('span')) > 3 else ''
        result[f'Hearing {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Hearing {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''
        result[f'Hearing {i+1} Result'] = result_text

    # Events (up to 25)
    event_blocks = soup.select('#filingInfo div.even, #filingInfo div.odd')[:25]
    for i, block in enumerate(event_blocks):
        desc = block.find('span', string=lambda s: s and 'Event' in s)
        date = block.find('span', string='Date Added:')
        result[f'Event {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Event {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''

    return result

# Scrape the sample
results = []
for _, row in tqdm(sampled_cases.iterrows(), total=len(sampled_cases), desc="Scraping eviction cases"):
    case_number = str(row['Case Number'])
    court_number = row['court_number']
    scraped = scrape_eviction_case(case_number, court_number)
    results.append(scraped)
    time.sleep(0.5)

# Save output
df_scraped = pd.DataFrame(results)
df_scraped.to_csv('Eviction_Cases_Sample_WithCourt.csv', index=False)
print("✅ Scraping complete. Saved to 'Eviction_Cases_Sample_WithCourt.csv'")


C:\Users\ferna\AppData\Local\Temp\ipykernel_14520\3087638131.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cases = pd.read_csv('Harris_EV_Filings_2023_042025_CLEAN.csv', usecols=['Case Number', 'court_number'])
Scraping eviction cases: 100%|█████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.40it/s]

✅ Scraping complete. Saved to 'Eviction_Cases_Sample_WithCourt.csv'


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import os

# Load your dataset and sample 100
df_cases = pd.read_csv('Harris_EV_Filings_2023_042025_CLEAN.csv', usecols=['Case Number', 'court_number'])
sampled_cases = df_cases.dropna(subset=['Case Number']).sample(n=100, random_state=42)

session = requests.Session()
output_file = 'Eviction_Cases_Sample_WithCourt.csv'

# Remove old file if re-running
if os.path.exists(output_file):
    os.remove(output_file)

def scrape_eviction_case(case_number, court_number):
    url = f'https://jpwebsite.harriscountytx.gov/CaseInfo/GetCaseInfo?case={case_number}'
    for attempt in range(3):
        try:
            response = session.get(url, timeout=10)
            response.raise_for_status()
            break
        except requests.RequestException as e:
            if attempt == 2:
                return {'Case Number': case_number, 'court_number': court_number, 'Error': str(e)}
            time.sleep(1)

    soup = BeautifulSoup(response.text, 'html.parser')
    result = {'Case Number': case_number, 'court_number': court_number}

    def extract_field(label):
        tag = soup.find('span', string=label)
        if tag and tag.find_next_sibling('span'):
            return tag.find_next_sibling('span').text.strip()
        return ''

    result['Filed Date'] = extract_field('Filed Date:')
    result['Case Status'] = extract_field('Case Status:')
    result['Nature of Claim'] = extract_field('Nature of Claim:')
    result['Disposition Desc'] = extract_field('Disposition:')
    result['Disposition Date'] = extract_field('Disposition Date:')
    result['Judgment Date'] = extract_field('Judgment Date:')
    result['Claim Amount'] = extract_field('Claim Amount:')

    # Parties
    party_blocks = soup.select('#partyInfo div.even, #partyInfo div.odd')
    for block in party_blocks:
        role = block.find('span', string='Party Type:')
        name = block.find('span', string='Party Name:')
        if role and name:
            role_value = role.find_next_sibling('span').text.strip()
            name_value = name.find_next_sibling('span').text.strip()
            if role_value == 'Plaintiff':
                result['Plaintiff'] = name_value
            elif role_value == 'Defendant':
                result['Defendant'] = name_value

    if not result.get('Filed Date') or not result.get('Plaintiff'):
        result['Error'] = 'Missing core data'
        return result

    # Hearings (up to 10)
    hearing_blocks = soup.select('#eventInfo div.even, #eventInfo div.odd')[:10]
    for i, block in enumerate(hearing_blocks):
        desc = block.find('span', string='Hearing Description:')
        date = block.find('span', string='Hearing Date/Time:')
        result_text = block.find_all('span')[-3].text.strip() if len(block.find_all('span')) > 3 else ''
        result[f'Hearing {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Hearing {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''
        result[f'Hearing {i+1} Result'] = result_text

    # Events (up to 25)
    event_blocks = soup.select('#filingInfo div.even, #filingInfo div.odd')[:25]
    for i, block in enumerate(event_blocks):
        desc = block.find('span', string=lambda s: s and 'Event' in s)
        date = block.find('span', string='Date Added:')
        result[f'Event {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Event {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''

    return result

# Scrape in chunks of 25
chunk_size = 25
for i in range(0, len(sampled_cases), chunk_size):
    chunk = sampled_cases.iloc[i:i+chunk_size]
    chunk_results = []
    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Processing chunk {i//chunk_size + 1}"):
        case_number = str(row['Case Number'])
        court_number = row['court_number']
        data = scrape_eviction_case(case_number, court_number)
        chunk_results.append(data)
        time.sleep(0.5)
    
    df_chunk = pd.DataFrame(chunk_results)
    df_chunk.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))

print(f"✅ All chunks scraped and saved to '{output_file}'")


C:\Users\ferna\AppData\Local\Temp\ipykernel_14520\283637755.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cases = pd.read_csv('Harris_EV_Filings_2023_042025_CLEAN.csv', usecols=['Case Number', 'court_number'])
Processing chunk 4: 100%|██████████████████████████████████████████████████████████████| 25/25 [00:18<00:00,  1.33it/s]

✅ All chunks scraped and saved to 'Eviction_Cases_Sample_WithCourt.csv'


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import os

# Load data and apply filters
df_all = pd.read_csv('Harris_EV_Filings_2023_042025_CLEAN.csv', usecols=['Case Number', 'court_number', 'Case File Date', 'Case Status'], low_memory=False)
df_all['Case File Date'] = pd.to_datetime(df_all['Case File Date'], errors='coerce')

# Keep only cases from 2024 and 2025 that are not 'Active'
df_filtered = df_all[
    (df_all['Case File Date'].dt.year >= 2024) &
    (df_all['Case File Date'].dt.year <= 2025) &
    (df_all['Case Status'].str.strip().str.lower() != 'active')
].dropna(subset=['Case Number']).drop_duplicates(subset=['Case Number'])

session = requests.Session()
output_file = 'Eviction_Cases_ByCourt_Sample20k.csv'

if os.path.exists(output_file):
    os.remove(output_file)

def scrape_eviction_case(case_number, court_number):
    url = f'https://jpwebsite.harriscountytx.gov/CaseInfo/GetCaseInfo?case={case_number}'
    for attempt in range(3):
        try:
            response = session.get(url, timeout=10)
            response.raise_for_status()
            break
        except requests.RequestException as e:
            if attempt == 2:
                return {'Case Number': case_number, 'court_number': court_number, 'Error': str(e)}
            time.sleep(1)

    soup = BeautifulSoup(response.text, 'html.parser')
    result = {'Case Number': case_number, 'court_number': court_number}

    def extract_field(label):
        tag = soup.find('span', string=label)
        if tag and tag.find_next_sibling('span'):
            return tag.find_next_sibling('span').text.strip()
        return ''

    result['Filed Date'] = extract_field('Filed Date:')
    result['Case Status'] = extract_field('Case Status:')
    result['Nature of Claim'] = extract_field('Nature of Claim:')
    result['Disposition Desc'] = extract_field('Disposition:')
    result['Disposition Date'] = extract_field('Disposition Date:')
    result['Judgment Date'] = extract_field('Judgment Date:')
    result['Claim Amount'] = extract_field('Claim Amount:')

    party_blocks = soup.select('#partyInfo div.even, #partyInfo div.odd')
    for block in party_blocks:
        role = block.find('span', string='Party Type:')
        name = block.find('span', string='Party Name:')
        if role and name:
            role_value = role.find_next_sibling('span').text.strip()
            name_value = name.find_next_sibling('span').text.strip()
            if role_value == 'Plaintiff':
                result['Plaintiff'] = name_value
            elif role_value == 'Defendant':
                result['Defendant'] = name_value

    if not result.get('Filed Date') or not result.get('Plaintiff'):
        result['Error'] = 'Missing core data'
        return result

    hearing_blocks = soup.select('#eventInfo div.even, #eventInfo div.odd')[:10]
    for i, block in enumerate(hearing_blocks):
        desc = block.find('span', string='Hearing Description:')
        date = block.find('span', string='Hearing Date/Time:')
        result_text = block.find_all('span')[-3].text.strip() if len(block.find_all('span')) > 3 else ''
        result[f'Hearing {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Hearing {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''
        result[f'Hearing {i+1} Result'] = result_text

    event_blocks = soup.select('#filingInfo div.even, #filingInfo div.odd')[:25]
    for i, block in enumerate(event_blocks):
        desc = block.find('span', string=lambda s: s and 'Event' in s)
        date = block.find('span', string='Date Added:')
        result[f'Event {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Event {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''

    return result

# Process per court
chunk_size = 500
grouped = df_filtered.groupby('court_number')

for court, group_df in grouped:
    print(f"\n📂 Processing court {court} — {len(group_df)} eligible cases")
    court_sample = group_df.sample(n=min(1500, len(group_df)), random_state=42)

    for i in range(0, len(court_sample), chunk_size):
        chunk = court_sample.iloc[i:i+chunk_size]
        chunk_results = []

        for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Court {court} | Chunk {i//chunk_size + 1}"):
            case_number = str(row['Case Number'])
            court_number = row['court_number']
            data = scrape_eviction_case(case_number, court_number)
            chunk_results.append(data)
            time.sleep(0.5)

        df_chunk = pd.DataFrame(chunk_results)
        df_chunk.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))

print(f"\n✅ Scraping complete. All results saved to '{output_file}'")



📂 Processing court JP 1-1 — 12146 eligible cases


Court JP 1-1 | Chunk 1:   3%|█▊                                                       | 16/500 [00:13<06:35,  1.22it/s]


KeyboardInterrupt: 

In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import os

# Load data and apply filters
df_all = pd.read_csv('Harris_EV_Filings_2023_042025_CLEAN.csv', usecols=['Case Number', 'court_number', 'Case File Date', 'Case Status', 'Disposition Desc', 'Disposition Date', 'Judgment Date', 'Claim Amount'], low_memory=False)
df_all['Case File Date'] = pd.to_datetime(df_all['Case File Date'], errors='coerce')

# Keep only cases from 2024 and 2025 that are not 'Active'
df_filtered = df_all[
    (df_all['Case File Date'].dt.year >= 2024) &
    (df_all['Case File Date'].dt.year <= 2025) &
    (df_all['Case Status'].str.strip().str.lower() != 'active')
].dropna(subset=['Case Number']).drop_duplicates(subset=['Case Number'])

session = requests.Session()
output_file = 'Eviction_Cases_ByCourt_Sample20k.csv'

if os.path.exists(output_file):
    os.remove(output_file)

def scrape_eviction_case(case_number, court_number, case_status, disp_desc, disp_date, judgment_date, claim_amount):
    url = f'https://jpwebsite.harriscountytx.gov/CaseInfo/GetCaseInfo?case={case_number}'
    for attempt in range(3):
        try:
            response = session.get(url, timeout=10)
            response.raise_for_status()
            break
        except requests.RequestException as e:
            if attempt == 2:
                return {'Case Number': case_number, 'court_number': court_number, 'Error': str(e)}
            time.sleep(1)

    soup = BeautifulSoup(response.text, 'html.parser')
    result = {'Case Number': case_number, 'court_number': court_number,
              'Case Status': case_status, 'Disposition Desc': disp_desc,
              'Disposition Date': disp_date, 'Judgment Date': judgment_date,
              'Claim Amount': claim_amount}

    # Scrape Hearings
    hearing_blocks = soup.select('#eventInfo div.even, #eventInfo div.odd')[:10]
    for i, block in enumerate(hearing_blocks):
        desc = block.find('span', string='Hearing Description:')
        date = block.find('span', string='Hearing Date/Time:')
        result_text = block.find_all('span')[-3].text.strip() if len(block.find_all('span')) > 3 else ''
        result[f'Hearing {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Hearing {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''
        result[f'Hearing {i+1} Result'] = result_text

    # Scrape Events
    event_blocks = soup.select('#filingInfo div.even, #filingInfo div.odd')[:25]
    for i, block in enumerate(event_blocks):
        desc = block.find('span', string=lambda s: s and 'Event' in s)
        date = block.find('span', string='Date Added:')
        result[f'Event {i+1} Description'] = desc.find_next_sibling('span').text.strip() if desc else ''
        result[f'Event {i+1} Date'] = date.find_next_sibling('span').text.strip() if date else ''

    return result

# Process per court
chunk_size = 500
grouped = df_filtered.groupby('court_number')

for court, group_df in grouped:
    print(f"\n📂 Processing court {court} — {len(group_df)} eligible cases")
    court_sample = group_df.sample(n=min(1500, len(group_df)), random_state=42)

    for i in range(0, len(court_sample), chunk_size):
        chunk = court_sample.iloc[i:i+chunk_size]
        chunk_results = []

        for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Court {court} | Chunk {i//chunk_size + 1}"):
            data = scrape_eviction_case(
                row['Case Number'], row['court_number'],
                row['Case Status'], row['Disposition Desc'],
                row['Disposition Date'], row['Judgment Date'], row['Claim Amount']
            )
            chunk_results.append(data)
            time.sleep(0.5)

        df_chunk = pd.DataFrame(chunk_results)
        df_chunk.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))

print(f"\n✅ Scraping complete. All results saved to '{output_file}'")



📂 Processing court JP 1-1 — 12146 eligible cases


Court JP 1-1 | Chunk 1: 100%|████████████████████████████████████████████████████████| 500/500 [06:19<00:00,  1.32it/s]
Court JP 1-1 | Chunk 2: 100%|████████████████████████████████████████████████████████| 500/500 [06:19<00:00,  1.32it/s]
Court JP 1-1 | Chunk 3: 100%|████████████████████████████████████████████████████████| 500/500 [06:15<00:00,  1.33it/s]



📂 Processing court JP 1-2 — 1304 eligible cases


Court JP 1-2 | Chunk 1: 100%|████████████████████████████████████████████████████████| 500/500 [06:14<00:00,  1.33it/s]
Court JP 1-2 | Chunk 2: 100%|████████████████████████████████████████████████████████| 500/500 [06:21<00:00,  1.31it/s]
Court JP 1-2 | Chunk 3: 100%|████████████████████████████████████████████████████████| 304/304 [03:47<00:00,  1.33it/s]



📂 Processing court JP 2-1 — 3558 eligible cases


...


📂 Processing court JP 8-1 — 3767 eligible cases


Court JP 8-1 | Chunk 1: 100%|████████████████████████████████████████████████████████| 500/500 [06:23<00:00,  1.30it/s]
Court JP 8-1 | Chunk 2: 100%|████████████████████████████████████████████████████████| 500/500 [06:16<00:00,  1.33it/s]
Court JP 8-1 | Chunk 3: 100%|████████████████████████████████████████████████████████| 500/500 [06:16<00:00,  1.33it/s]



📂 Processing court JP 8-2 — 1669 eligible cases


Court JP 8-2 | Chunk 1: 100%|████████████████████████████████████████████████████████| 500/500 [06:16<00:00,  1.33it/s]
Court JP 8-2 | Chunk 2: 100%|████████████████████████████████████████████████████████| 500/500 [06:09<00:00,  1.35it/s]
Court JP 8-2 | Chunk 3: 100%|████████████████████████████████████████████████████████| 500/500 [06:08<00:00,  1.36it/s]


✅ Scraping complete. All results saved to 'Eviction_Cases_ByCourt_Sample20k.csv'
